In [1]:
# Concatenate all text files

import os
import pandas as pd

def concatenate_csv_files(output_file_path):
    # Get a list of all CSV files in the current directory
    csv_files = [file for file in os.listdir() if file.endswith('.csv')]

    # Load the CSV files into pandas dataframes
    dataframes = []
    for file in csv_files:
        df = pd.read_csv(file)
        dataframes.append(df)

    # Concatenate the dataframes
    concatenated_df = pd.concat(dataframes, axis=0)

    # If the output file exists, ask the user whether to overwrite it
    if os.path.exists(output_file_path):
        user_input = input(f"The file {output_file_path} already exists. Do you want to overwrite it? (y/n)")
        if user_input.lower() == 'n':
            return

    # Write the concatenated dataframe to a new CSV file
    concatenated_df.to_csv(output_file_path, index=False)
    print(f"Concatenated file saved to {output_file_path}")


concatenate_csv_files('titles2017-20230420.csv')


Concatenated file saved to titles2017-20230420.csv


In [1]:
# Remove non alpha numeric characters and extra bits of text grabbed

import pandas as pd
import re

def clean_titles(csv_file_path):
    # Load the CSV file into a pandas dataframe
    df = pd.read_csv(csv_file_path)

    # Clean the 'title' column
    df['title'] = df['title'].apply(lambda x: re.sub(r'\([^()]*\)', '', str(x))) # remove text enclosed in parentheses
    df['title'] = df['title'].apply(lambda x: re.sub(r'[^\w\s()]|(?<=\()\w+|\w+(?=\))', '', str(x)))

    # Print the cleaned dataframe
    df.to_csv('sentiment_analysis_results.csv',index=False)

clean_titles('sentiment_analysis_results.csv')



In [7]:
#Remove all duplicate titles 

import pandas as pd

# Read in the CSV file
df = pd.read_csv('titles2017-20230420Clean.csv')

# Drop duplicate rows based on the "title" column
df.drop_duplicates(subset="title", inplace=True)

# Save the cleaned data to a new CSV file
df.to_csv('titles2017-20230420Clean.csv', index=False)

In [8]:
# Sort by date

import pandas as pd

def sort_csv_file_by_date(csv_file_path):
    # Load the CSV file into a pandas dataframe
    df = pd.read_csv(csv_file_path)

    # Sort the dataframe by the 'date' column
    sorted_df = df.sort_values('date', ascending=True)

    # Print the sorted dataframe
    sorted_df.to_csv('titles2017-20230420Clean.csv', index=False)

sort_csv_file_by_date('titles2017-20230420Clean.csv')

In [6]:
from transformers import pipeline
import pandas as pd
from transformers import pipeline

# Load sentiment analysis model
specific_model = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")
specific_model = pipeline("sentiment-analysis")
data = ["I love you", "I hate you"]
specific_model(data)
# Read in CSV file
df = pd.read_csv("titles2017-20230420.csv")
df.dropna(subset=['title'], inplace=True)

titles = list(df['title'].astype(str))
sentiments = specific_model(titles)

# Perform sentiment analysis on each title
# sentiments = specific_model(list(df["title"]))

# Extract sentiment polarity and subjectivity
polarity = [s["score"] for s in sentiments]
Plabel = [s["label"] for s in sentiments]

# Add new columns to dataframe
df["TPolarity"] = polarity
df["TLabel"] = Plabel

# Write results to new CSV file
df.to_csv("sentiment_analysis_results.csv", index=False)




emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


: 

: 

In [4]:
from transformers import pipeline
import pandas as pd
from transformers import pipeline

# Load sentiment analysis model
specific_model = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")
specific_model = pipeline("sentiment-analysis")
data = ["I love you", "I hate you"]
specific_model(data)

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998656511306763},
 {'label': 'NEGATIVE', 'score': 0.9991129040718079}]

In [8]:
# Add in 1 or 0 for positive or negative 
import pandas as pd
import re

def clean_titles(csv_file_path):
    # Load the CSV file into a pandas dataframe
    df = pd.read_csv(csv_file_path)

    df['label'] = df['TLabel'].apply(lambda x: 1 if x == 'POSITIVE' else 0)

    df.to_csv('SATest.csv',index=False)

clean_titles('sentiment_analysis_results - Copy.csv')

In [9]:
# Now output files for training transformer so it will be json with text and 0 or 1 for pos/negativity
import json

def convert_to_json(csv_file_path, json_file_path):
    # Load the CSV file into a pandas dataframe
    df = pd.read_csv(csv_file_path)

    # Create a list of JSON objects, one for each row in the dataframe
    json_list = []
    for index, row in df.iterrows():
        json_obj = {'label': row['label'], 'text': row['title']}
        json_list.append(json_obj)

    # Write the list of JSON objects to a text file
    with open(json_file_path, 'w') as f:
        for json_obj in json_list:
            json.dump(json_obj, f)
            f.write('\n')

convert_to_json('SATest.csv', 'titles_training.jsonl')


In [10]:
# Alternatively create new file with only the title and lable rows

import pandas as pd

def extract_columns(csv_file_path, new_csv_file_path):
    # Load the CSV file into a pandas dataframe
    df = pd.read_csv(csv_file_path)

    # Extract the 'title' and 'TLabel' columns
    df = df[['title', 'label']]

    # Output the new CSV file with only the selected columns
    df.to_csv(new_csv_file_path, index=False)

extract_columns('SATest.csv', 'SATestSimple.csv')




In [13]:
# Similar process above but produces training set with polarity values
import pandas as pd

def preprocess_csv(input_file_path, output_file_path):
    # Read in the CSV file into a pandas dataframe
    df = pd.read_csv(input_file_path)
    # Negate the 'TPolarity' value if 'TLabel' is 'NEGATIVE'

    df.loc[df['TLabel'] == 'NEGATIVE', 'TPolarity'] = -1 * df.loc[df['TLabel'] == 'NEGATIVE', 'TPolarity']
    # Select only the 'title' and 'TPolarity' columns


    df = df[['title', 'TPolarity']]
    # Save the resulting dataframe to a new CSV file
    df.to_csv(output_file_path, index=False)

# Example usage:
preprocess_csv('SATest.csv', 'SATTestSimple1.0.csv')